In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LarsCV, ElasticNet, ElasticNetCV, LinearRegression, LassoCV
# import statsmodels.api as sm
# from pandas_profiling import ProfileReport

In [ ]:
# df1 = pd.read_csv('../input/graduate-admissions/Admission_Predict.csv')
# df2 = pd.read_csv('../input/graduate-admissions/Admission_Predict_Ver1.1.csv')
df  = pd.read_csv('./Admission_Prediction.csv')

In [ ]:

df.head()


In [ ]:
df['University Rating'].mode()

In [ ]:
pf = ProfileReport(df)
pf.to_widgets()


In [ ]:
# pf_df1 = ProfileReport(df_1)
# pf_df1.to_widgets()

In [ ]:
print(df.columns)
print(df.isnull().sum())
df['GRE Score']         = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score']       = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mode()[0])
print(df.isnull().sum())

In [ ]:
x = df[['GRE Score', 'TOEFL Score', 'University Rating', 'SOP','LOR', 'CGPA', 'Research']]
y = df[['Chance of Admit']]

In [ ]:
sc     = StandardScaler()
data_x = sc.fit_transform(x)
x_std  = pd.DataFrame(data_x,columns=x.columns)
x_std.head()

In [ ]:
data_x.shape

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [ ]:
x_vif = [vif(data_x, i) for i in range(0,data_x.shape[1])]
df_vif = pd.DataFrame(x_vif, columns = ['VIF'], index = x_std.columns)
df_vif

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_std,y,random_state=42,test_size=0.25)

In [ ]:
X_train.head()

In [ ]:
lr_model = LinearRegression(n_jobs=-1)

In [ ]:
lr_model.fit(X_train,y_train)

In [ ]:
y_pred = lr_model.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
print("MAE = ", metrics.mean_absolute_error(y_test,y_pred))

In [ ]:
print("Score = ", lr_model.score(X_test,y_test))
# lr_model.score()

In [ ]:
def adj_r2(model,x,y):
    r2 = model.score(x,y)  # r-squared value
    n  = x.shape[0]     # Number of rows
    p  = x.shape[1]     # Number of features
    adjusted_r2 = 1 - (1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [ ]:
adj_r2(lr_model, X_test, y_test)

In [ ]:
lassocv = LassoCV(cv=20,
                 max_iter=1000,
                 normalize=True)

In [ ]:
lassocv.fit(X_train, y_train)

In [ ]:
lassocv.alpha_

In [ ]:
lasso_model = Lasso(alpha=lassocv.alpha_)

In [ ]:
lasso_model.fit(X_train,y_train)

In [ ]:
lasso_model.score(X_test,y_test)

In [ ]:
ridge_cv = RidgeCV(alphas=np.random.uniform(0,10,50),
                   cv = 10,
                   normalize = True)

In [ ]:
ridge_cv.fit(X_train, y_train)
ridge_cv.alpha_

In [ ]:
ridge_model = Ridge(alpha=ridge_cv.alpha_,
                   random_state=42)
ridge_model.fit(X_train,y_train)

In [ ]:
ridge_model.score(X_test, y_test)

In [ ]:
elasticnet_cv = ElasticNetCV(cv=20)
elasticnet_cv.fit(X_train,y_train)
elasticnet_cv.alpha_

In [ ]:
elasticnet_cv.l1_ratio

In [ ]:
elastic = ElasticNet(alpha=elasticnet_cv.alpha_,
                    l1_ratio=elasticnet_cv.l1_ratio,
                    #normalize=True,
                    random_state=42)
elastic.fit(X_train,y_train)
elastic.score(X_test,y_test)

In [ ]:
X_to_predict = [[337.0, 118.0, 2.0, 4.0, 3.0, 9.65, 1]]
X_to_predict = sc.fit_transform(X_to_predict)
print(X_to_predict[0])
print(elastic.predict(X_to_predict))

In [ ]:
pickle.dump(lr_model,open("admis_pred_model.pickle",'wb'))

In [ ]:
!ls